In [1]:
from slokabase.SqliteModel import get_create_table_query, get_insert_query, get_read_query, get_update_query, get_delete_query
from slokabase.SqliteModel import SqliteModel

In [2]:
import os, sqlite3 
# db_name = 'slokabase_10.db'
db_name = 'slokabase.db'
db_path = os.path.join(os.getcwd(),db_name)

In [3]:
SongIndex_sql = SqliteModel(db_path,'SongIndex')
mySongs_sql = SqliteModel(db_path,'Songs')

In [4]:
db_name = 'dictionary.db'
db_path = os.path.join(os.getcwd(),db_name)
DictonaryWord_sql = SqliteModel(db_path,'DictWord')
DictonaryMeaning_sql = SqliteModel(db_path,'DictMeaning')

In [5]:
DictonaryWord_sql.create_table(word ='VARCHAR(300) NOT NULL UNIQUE',
                    phonetics = 'VARCHAR(300)',
                    word_idx='INTEGER AUTO INCREMENT', 
                    PRIMARY= 'KEY (word)')


Table : DictWord Created Successful !


In [18]:
# SongIndex_sql.create_entry(**songIdx_info)
# DictonaryWord_sql.create_entry(word='ākta')
# print('asd')
for word_temp in  ['kṣitiḥ','iha',  'vipulatare' ,'tiṣṭhati', 'tava' ,'pṛṣṭhe']:
# dharaṇi-dhāraṇa=holding up the world; kiṇa-cakra=a large circular scar; gariṣṭhe=the heaviest;
# keśava=O Lord Keśava, of fine hair!; dhṛta-kūrma-śarīra=who have accepted the form of a tortoise;
# ['keśava','dhṛtamīna-śarīra','jaya']:
# ['pralaya-payodhi-jale','dhṛtavān', 'asi', 'vedam']:
    DictonaryWord_sql.create_entry(word=word_temp)
    


In [6]:
DictonaryMeaning_sql.create_table(word='VARCHAR(300)', 
                    meaning_idx='INTEGER NOT NULL', 
                    meaning_value='TEXT NOT NULL',
                    meaning_type= 'VARCHAR(100)', 
                    reference='TEXT',
                    example='TEXT', 
                    synonyms= 'TEXT',      
                    source= 'TEXT',  
                    FOREIGN = 'KEY (word) REFERENCES DictonaryWord_sql(word)',
                    PRIMARY= 'KEY (word, meaning_value)')

# grammer, singular/pular, gender,part of speech, 1st, 2nd, 3rd person
# like hypter link bg1.2,sb 1.2.4, song_id.verse_no
# similar synonyms
# hyper links

Table : DictMeaning Created Successful !


In [10]:
word_temp ='iha' # 'veda'
word_meaning = 'not my knows'
# ref ='ŚriDaśavātāraStotraṁ/1'
ref ='ŚriDaśavātāraStotraṁ/4'
def create_entry_dic(word_temp,word_meaning,ref,DictonaryWord_sql,DictonaryMeaning_sql):
    if len(DictonaryWord_sql.read_entry(word=word_temp)) == 0:
        print(f'Create Entry of word="{word_temp}" in DictWord Table')
        DictonaryWord_sql.create_entry(word=word_temp)
    else:
        print(f'Not Unique, word_temp="{word_temp}" is already present in DictWord Table')
    
    if len(DictonaryMeaning_sql.read_entry(word=word_temp,meaning_value=word_meaning)) ==1:
        print(f'Not Unique, word meaning="{word_meaning}" is already present in Dict Meaning Table')
        pre_ref = DictonaryMeaning_sql.read_entry(word=word_temp,meaning_value=word_meaning)[0]['reference']
        if ref not in pre_ref.split(','):
            print('need to update')
            new_ref = pre_ref.split(',')
            new_ref.append(ref)
            new_ref=','.join(new_ref)
            # print(new_ref)
            # print(','.join(new_ref))
            DictonaryMeaning_sql.update_entry(*['word','meaning_value'],reference=new_ref,word=word_temp,meaning_value=word_meaning)
            print(f'Updated refence: added new ref {ref} to Dict Meaning Table ')
        else:
            print(f'Not Unique, refence="{pre_ref}" is already present in Dict Meaning Table')
    else : 
        print(f'word meaning="{word_meaning}" is not present in Dict Meaning Table')
        last_idx = len(DictonaryMeaning_sql.read_entry(word=word_temp))
        DictonaryMeaning_sql.create_entry(word=word_temp,meaning_idx=last_idx+1,meaning_value=word_meaning,reference=ref)
        print(f'Create Entry of word meaning="{word_meaning}" in Dict Meaning Table')

# create_entry_dic(word_temp,word_meaning,ref,DictonaryWord_sql,DictonaryMeaning_sql)

# temp_dic['division'] = f'{ŚriDaśavātāraStotraṁ}/{}'

In [22]:
# song_idx = SongIndex_sql.read_entry(*['song_idx', 'song_name','song_short_name'])[0]
for song_idx in SongIndex_sql.read_entry(*['song_idx', 'song_name','song_short_name']):
    # {'song_idx': 1,
    #  'song_name': 'Śri Daśavātāra Stotraṁ',
    #  'song_short_name': 'ŚriDaśavātāraStotraṁ'}
    for each_sloka in mySongs_sql.read_entry(song_idx=song_idx['song_idx']):
        reference = f"{song_idx['song_short_name']}/{each_sloka['slokas_no']}"
        # print()
        if type(each_sloka['synonyms']) is str:
            for each_line in each_sloka['synonyms'].split('\n'):
                for item_string in each_line.split(';'):
                    if len(item_string.split('='))==2:
                        pairs = item_string.split('=')
                        print(f"   {pairs[0].strip()}= {pairs[1].strip()}")
                        create_entry_dic(pairs[0].strip(),pairs[1].strip(),reference,DictonaryWord_sql,DictonaryMeaning_sql)        
# mySongs_sql.read_entry(song_idx=song_idx['song_idx'])

   pralaya-payodhi-jale= in the turbulent waters of devastation
Not Unique, word_temp="pralaya-payodhi-jale" is already present in DictWord Table
Not Unique, word meaning="in the turbulent waters of devastation" is already present in Dict Meaning Table
Not Unique, refence="ŚriDaśavātāraStotraṁ/1" is already present in Dict Meaning Table
   dhṛtavān= holding
Not Unique, word_temp="dhṛtavān" is already present in DictWord Table
Not Unique, word meaning="holding" is already present in Dict Meaning Table
Not Unique, refence="ŚriDaśavātāraStotraṁ/1" is already present in Dict Meaning Table
   asi= You are
Not Unique, word_temp="asi" is already present in DictWord Table
Not Unique, word meaning="You are" is already present in Dict Meaning Table
Not Unique, refence="ŚriDaśavātāraStotraṁ/1" is already present in Dict Meaning Table
   vedam= the Vedas
Not Unique, word_temp="vedam" is already present in DictWord Table
Not Unique, word meaning="the Vedas" is already present in Dict Meaning Tabl

UnboundLocalError: cannot access local variable 'data' where it is not associated with a value

In [24]:
# last_idx = len(DictonaryMeaning_sql.read_entry(word=word_temp))
# DictonaryMeaning_sql.create_entry(word=word_temp,meaning_idx=last_idx+1,meaning_value=word_meaning)
# DictonaryMeaning_sql.read_entry(*['meaning_value'],word=word_temp)
# DictonaryMeaning_sql.read_entry(*['meaning_value'],word=word_temp,meaning_value=word_meaning)
song_idx['song_idx']
    # print('ad')
print(reference,pairs[0].strip(),pairs[1].strip())

Mahiṣāsura Mardini Stotraṁ/1 ayi As a gentle address in the sense of 'submissive' 'friend',I pray, also of encouragement or persuation,kind inquiry


In [14]:
# DictonaryMeaning_sql.delete_entry(word='veda',meaning_value='knows')
# DictonaryMeaning_sql.delete_entry(meaning_idx=2)
DictonaryMeaning_sql.delete_entry(meaning_idx=1)

DELETE FROM DictMeaning WHERE meaning_idx=1;


In [130]:

# select word, meaning_value,meaning_idx,reference from DictMeaning ORDER BY word, meaning_value ASC;

# select word, meaning_value,meaning_idx,reference from DictMeaning  where word='kṛṣṇa'ORDER BY word, meaning_value ASC;
# db_name = 'slokabase.db'
db_name = 'dictionary.db'
def get_all_dict_words(db_name):
    db_path = os.path.join(os.getcwd(),db_name)
    db_connect = sqlite3.connect(db_path)
    db_cursor = db_connect.cursor()
    # query = f"""select word, meaning_value,reference from DictMeaning  where word='{dic_word}' ORDER BY word, meaning_value ASC;"""
    # query = f"""select word from DictMeaning  ORDER BY word COLLATE NOCASE, meaning_value ASC;"""
    query = f"""select word from DictWord  ORDER BY word COLLATE NOCASE ASC;"""
    db_cursor.execute(query)
    data = db_cursor.fetchall()
    db_cursor.close()
    # print(data)
    order_list = []
    for i in data:
        # print()
        if i[0][0] not in order_list:
            order_list.append(i[0])
    # print(order_list)
    return order_list

all_dict_word =get_all_dict_words('dictionary.db')
print(all_dict_word)

['(kabe)', '(tāra)', '(āmār)', '(āmār) nayaneri', 'ab', 'abdhi-magnam', 'abhaya', 'abhaya-caraṇāravinda', 'abhibhūtau', 'abhilāṣ', 'abhilāṣī', 'abhimarśa', 'abhimāna', 'abhinava', 'abhīṣṭa-pūraṇ', 'abja', 'abṛta', 'acintya', 'adarśanāt', 'adbhuta-vāmana', 'adbhuta-śṛńgam', 'adhama', 'adhama janāra', 'adhama-tāraṇ', 'adharam', 'adhi', 'adhika', 'adhikārī', 'advaita gosāñi', 'agam', 'agha-nāśa-i', 'ahaha', 'ahaḥ-niśam', 'ahaitukī', 'ahaitukī bhakti', 'aham', 'ahaḿ', 'ahańkāre', 'ahita', 'aise', 'aisī', 'ajani', 'ajñam', 'ajñāna', 'ajñāna jāni', 'akhedam', 'akhila', 'akhilam', 'akrodha', 'akṣidṛśyaḥ', 'akṣobhya paramahaḿse', 'akṣobhyera śiṣya', 'akāri', 'akārye', 'alakoń', 'alam', 'alasa hoy', 'amala', 'amala harinām', 'amani', 'amani jāge', 'ambhaḥ-nidhī', 'ambudhi', 'amiya', 'amiya pariche khasi', 'amiya vilāsā', 'amla', 'amṛta rasāla', 'amṛta-keli', 'amāninā', 'amānī hoye', 'anala', 'anale paśibo', 'ananta', 'ananta-līlāya', 'ananya', 'andha', 'aneka', 'aneka jatan

In [223]:
# 'a' : 'a ā ’'
# 'e' : 'e ê ē'
# 'i' : 'i ī'
# 'o' : 'o ô ō' 
# 'u' : 'u ū'
# 'r' : 'r r̥ r̥̄'
# 'l' : 'l l̥ l̥̄'
# 'h' : 'h ḥ'
# 'm' : 'm ṁ'
# 'n' : 'n ṅ ñ ṇ'
# 's' : 's ś ṣ'
# 't' : 't t ṭ'
# 'd' : 'd d ḍ'

# # vowels 
# k kh
# g gh

all_dict_word

# pattern = re.compile(r'^abc') # match word/char starting with 
# pattern = re.compile(r'xyz$') # match word/char ending with 


# import re

        # string to raw_string: https://www.geeksforgeeks.org/python-raw-strings/
    # raw_string = repr(string_defined)[1:-1]
# patter =  r'[aā]p'
# patter =  r'k[rṛ]'
# patter =  r'[sśṣ]'
patter =  r'.ń.'
# patter =  r'krsna'
print(patter)
r = re.compile(patter)
# matches = 
# r.findall(' '.join(all_dict_word))
newlist = list(filter(r.match, all_dict_word)) # Read Note below
print(newlist)

.ń.
['ańga', 'ańgīkara kore', 'ūńcī']


In [219]:
import re

text_2_search = """
abcdefghijklmnopqrstuvwxyz
ABCDEFGHIJKLMNOPQRSTUVWXYZ

Ha HaHa
apagatapāpam
aparādha
bhava-tāpam
bhuliyā āpana-dhana
MetaCharacters (Need to be escape)
. ^ $ * + { } [ ] \ | ()

abcasdas

coremys.com'

321-555-4321
123.555.1234
123*555*1234
800-555-4321
900-555-4321
Mr. Schafer
Mr Smith
Ms Davis

Mrs. Robinson
Mr. T


cat
batman
path
bat
match

CoreyMSchafer@gmail.com
corey.schafer@university.edu
corey-321-schafer@my-work.net
"""
# pattern = re.compile(r'\bHa')  # match two Ha HaHa starting 'Ha' as at starting
# pattern = re.compile(r'\BHa') # match last Ha because it has no word boundary
# pattern = re.compile(r'^abc') # match word/char starting with 
# pattern = re.compile(r'xyz$') # match word/char ending with 
# pattern = re.compile(r'\d\d\d.\d\d\d.\d\d\d\d')
# pattern = re.compile(r'\d{3}.\d{3}.\d{4}')
# pattern = re.compile(r'\d{3}[-.]\d{3}[-.]\d{4}') 
# pattern = re.compile(r'\d{3}.\d{3}.\d{4}')
# pattern = re.compile(r'[89]00.\d{3}.\d{4}')
# pattern = re.compile(r'[a-zA-Z]') # match char with a-z and A-Z
# pattern = re.compile(r'[^a-zA-Z]')  # match char which is not a-z and A-Z
pattern = re.compile(r'k')  # match which doesnot include bat 

# quantifiers
pattern.findall(text_2_search)

['k', 'k']

In [76]:
# data = db_cursor.fetchall()
order_list = []
for i in data:
    # print()
    if i[0][0] not in order_list:
        # print('append')
        # order_list.append(i[0][0])
        order_list.append(i[0])
print(order_list)

['(kabe)', '(tāra)', '(āmār)', '(āmār) nayaneri', 'ab', 'abdhi-magnam', 'abhaya', 'abhaya-caraṇāravinda', 'abhibhūtau', 'abhilāṣ', 'abhilāṣī', 'abhimarśa', 'abhimāna', 'abhinava', 'abhīṣṭa-pūraṇ', 'abja', 'abṛta', 'acintya', 'adarśanāt', 'adbhuta-vāmana', 'adbhuta-śṛńgam', 'adhama', 'adhama janāra', 'adhama-tāraṇ', 'adharam', 'adhi', 'adhika', 'adhikārī', 'advaita gosāñi', 'agam', 'agha-nāśa-i', 'ahaha', 'ahaḥ-niśam', 'ahaitukī', 'ahaitukī bhakti', 'aham', 'ahaḿ', 'ahańkāre', 'ahita', 'aise', 'aisī', 'ajani', 'ajñam', 'ajñāna', 'ajñāna jāni', 'akhedam', 'akhila', 'akhilam', 'akrodha', 'akṣidṛśyaḥ', 'akṣobhya paramahaḿse', 'akṣobhyera śiṣya', 'akāri', 'akārye', 'alakoń', 'alam', 'alasa hoy', 'amala', 'amala harinām', 'amani', 'amani jāge', 'ambhaḥ-nidhī', 'ambudhi', 'amiya', 'amiya pariche khasi', 'amiya vilāsā', 'amla', 'amṛta rasāla', 'amṛta-keli', 'amāninā', 'amānī hoye', 'anala', 'anale paśibo', 'ananta', 'ananta-līlāya', 'ananya', 'andha', 'aneka', 'aneka jatan

In [206]:
set_acc = set()
for i in order_list:
    set_acc =set_acc.union(set(i))

set_acc

{' ',
 '!',
 '(',
 ')',
 ',',
 '-',
 'L',
 'M',
 'O',
 'a',
 'b',
 'c',
 'd',
 'e',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'ñ',
 'ā',
 'ī',
 'ń',
 'ś',
 'ū',
 '́',
 '̄',
 '̇',
 '̣',
 'ḍ',
 'ḥ',
 'ṁ',
 'ṅ',
 'ṇ',
 'ṛ',
 'ṣ',
 'ṭ',
 '‘',
 '’'}

In [46]:
new_data = []
for tuple in data:
    temp_line = tuple 
    pi

[('tomāra', 'Your', 'emonadurmati/6,gopinath1/2,gopinath1/6,gurudeva4/4'),
 ('tomāra', 'is Your', 'gopinath1/4'),
 ('tomāra', 'is yours', 'ohevaisnava/4'),
 ('tomāra', 'of Yours', 'mamamanamandire/5'),
 ('tomāra', 'your', 'namonamahtulasikrsna1 /3'),
 ('tomāra', 'yours.', 'jivjago/3')]

In [65]:
new_data = []
for tuple_data in data:
    temp_line = list(tuple_data)
    ref_list = []

    for ref in temp_line[2].split(','):
        # print()
        ref_dic = dict()   
        # print(ref)
        temp_short_name = ref.split('/')[0]
        my_song_idx  = SongIndex_sql.read_entry(song_short_name=temp_short_name)[0]['song_idx']
        ref_dic[f"{my_song_idx}/{ref.split('/')[1]}"] = ref
        ref_list.append(ref_dic)
    temp_line[2] = ref_list
    new_data.append(temp_line)
    # print(ref_list)
new_data

[['tomāra',
  'Your',
  [{'17/6': 'emonadurmati/6'},
   {'22/2': 'gopinath1/2'},
   {'22/6': 'gopinath1/6'},
   {'25/4': 'gurudeva4/4'}]],
 ['tomāra', 'is Your', [{'22/4': 'gopinath1/4'}]],
 ['tomāra', 'is yours', [{'47/4': 'ohevaisnava/4'}]],
 ['tomāra', 'of Yours', [{'41/5': 'mamamanamandire/5'}]],
 ['tomāra', 'your', [{'69/3': 'namonamahtulasikrsna1 /3'}]],
 ['tomāra', 'yours.', [{'39/3': 'jivjago/3'}]]]

In [58]:
SongIndex_sql.read_entry(*['song_idx', 'song_name','song_short_name'])
SongIndex_sql.read_entry(song_short_name='GovindaDāmodaraMādhaveti')[0]['song_idx']

3

In [100]:
def get_single_dic(db_name,dic_word):
    db_name = 'dictionary.db'
    dic_word = 'tomāra'
    # dic_word = 'kṛṣṇa'
    db_path = os.path.join(os.getcwd(),db_name)

    db_connect = sqlite3.connect(db_path)
    db_cursor = db_connect.cursor()
    query = f"""select word, meaning_value,reference from DictMeaning  where word='{dic_word.strip()}' ORDER BY word, meaning_value ASC;"""
    db_cursor.execute(query)
    data = db_cursor.fetchall()
    db_cursor.close()
    return data
data = get_single_dic(db_name,dic_word)

In [126]:
def add_reference2single_dict(db_name,data):
    db_path = os.path.join(os.getcwd(),db_name)
    SongIndex_sql = SqliteModel(db_path,'SongIndex')
    dict_word = data[0][0]
    
    new_data = []
    # new_data.append(dict_word)
    # print('data inside fun add_ref2single_dict: ',data)
    for tuple_data in data:
        temp_line = list(tuple_data)
        del temp_line[0]
        # print()
        # print('Befor temp_line: ',temp_line)
        # print('temp_line :', temp_line)
        ref_list = []
        for ref in temp_line[1].split(','):
            ref_dic = dict()   
            # print(ref)
            temp_short_name = ref.split('/')[0]
            my_song_idx  = SongIndex_sql.read_entry(song_short_name=temp_short_name)[0]['song_idx']
            ref_dic[f"{my_song_idx}/{ref.split('/')[1]}"] = ref
            ref_list.append(ref_dic)
        temp_line[1] = ref_list
        # print('After temp_line : ',temp_line)
        # print()
        new_data.append(temp_line)
        data = [dict_word,new_data]
    # new_data.insert(0,dict_word)
    return data
add_reference2single_dict('slokabase.db',data)

['tomāra',
 [['Your',
   [{'17/6': 'emonadurmati/6'},
    {'22/2': 'gopinath1/2'},
    {'22/6': 'gopinath1/6'},
    {'25/4': 'gurudeva4/4'}]],
  ['is Your', [{'22/4': 'gopinath1/4'}]],
  ['is yours', [{'47/4': 'ohevaisnava/4'}]],
  ['of Yours', [{'41/5': 'mamamanamandire/5'}]],
  ['your', [{'69/3': 'namonamahtulasikrsna1 /3'}]],
  ['yours.', [{'39/3': 'jivjago/3'}]]]]

In [115]:
add_reference2single_dict('slokabase.db',data)

['tomāra',
 ['Your',
  [{'17/6': 'emonadurmati/6'},
   {'22/2': 'gopinath1/2'},
   {'22/6': 'gopinath1/6'},
   {'25/4': 'gurudeva4/4'}]],
 ['is Your', [{'22/4': 'gopinath1/4'}]],
 ['is yours', [{'47/4': 'ohevaisnava/4'}]],
 ['of Yours', [{'41/5': 'mamamanamandire/5'}]],
 ['your', [{'69/3': 'namonamahtulasikrsna1 /3'}]],
 ['yours.', [{'39/3': 'jivjago/3'}]]]

In [237]:
# for i in '0123456789':                    # ascii code range [48 57]
# for i in 'ABCDEFGHIJKLMNOMPQRSTUVWXYZ':   # ascii code range [65 96]    
# for i in 'abcdefghijklmnopqrstuvwxyz':    # ascii code range [97 112]
for i in 'a ā ’ i ī u ū ṛ e ē ai o au ṁ ḥ k kh g gh ṅñ c ch j jh ṭ ṭh ḍ ḍh ṇ t th d dh n ńśṣsṭr':
    print(f'{i} : {ord(i)}')

a : 97
ā : 257
’ : 8217
i : 105
ī : 299
u : 117
ū : 363
e : 101
ē : 275
a : 97
i : 105
ṛ : 7771
ḥ : 7717
ṅ : 7749
ñ : 241
ṇ : 7751
n : 110
ń : 324
ś : 347
ṣ : 7779
s : 115
ṭ : 7789
r : 114


In [240]:
for i in range(1,9790):
    print(chr(i) ,end=' ')

       	 
                     ! " # $ % & ' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ? @ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ \ ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z { | } ~                                    ¡ ¢ £ ¤ ¥ ¦ § ¨ © ª « ¬ ­ ® ¯ ° ± ² ³ ´ µ ¶ · ¸ ¹ º » ¼ ½ ¾ ¿ À Á Â Ã Ä Å Æ Ç È É Ê Ë Ì Í Î Ï Ð Ñ Ò Ó Ô Õ Ö × Ø Ù Ú Û Ü Ý Þ ß à á â ã ä å æ ç è é ê ë ì í î ï ð ñ ò ó ô õ ö ÷ ø ù ú û ü ý þ ÿ Ā ā Ă ă Ą ą Ć ć Ĉ ĉ Ċ ċ Č č Ď ď Đ đ Ē ē Ĕ ĕ Ė ė Ę ę Ě ě Ĝ ĝ Ğ ğ Ġ ġ Ģ ģ Ĥ ĥ Ħ ħ Ĩ ĩ Ī ī Ĭ ĭ Į į İ ı Ĳ ĳ Ĵ ĵ Ķ ķ ĸ Ĺ ĺ Ļ ļ Ľ ľ Ŀ ŀ Ł ł Ń ń Ņ ņ Ň ň ŉ Ŋ ŋ Ō ō Ŏ ŏ Ő ő Œ œ Ŕ ŕ Ŗ ŗ Ř ř Ś ś Ŝ ŝ Ş ş Š š Ţ ţ Ť ť Ŧ ŧ Ũ ũ Ū ū Ŭ ŭ Ů ů Ű ű Ų ų Ŵ ŵ Ŷ ŷ Ÿ Ź ź Ż ż Ž ž ſ ƀ Ɓ Ƃ ƃ Ƅ ƅ Ɔ Ƈ ƈ Ɖ Ɗ Ƌ ƌ ƍ Ǝ Ə Ɛ Ƒ ƒ Ɠ Ɣ ƕ Ɩ Ɨ Ƙ ƙ ƚ ƛ Ɯ Ɲ ƞ Ɵ Ơ ơ Ƣ ƣ Ƥ ƥ Ʀ Ƨ ƨ Ʃ ƪ ƫ Ƭ ƭ Ʈ Ư ư Ʊ Ʋ Ƴ ƴ Ƶ ƶ Ʒ Ƹ ƹ ƺ ƻ Ƽ ƽ ƾ ƿ ǀ ǁ ǂ ǃ Ǆ ǅ ǆ Ǉ ǈ ǉ Ǌ ǋ ǌ Ǎ ǎ Ǐ ǐ Ǒ ǒ Ǔ ǔ Ǖ ǖ Ǘ ǘ Ǚ ǚ Ǜ ǜ ǝ Ǟ ǟ Ǡ ǡ Ǣ ǣ Ǥ ǥ Ǧ ǧ Ǩ ǩ Ǫ ǫ Ǭ ǭ Ǯ ǯ ǰ Ǳ ǲ ǳ Ǵ ǵ Ƕ Ƿ Ǹ 